# BCS Testing

In [1]:
import numpy as np
from scipy.stats import qmc

import PyUQTk.uqtkarray as uqtkarray
import PyUQTk.pce as uqtkpce
import PyUQTk.PyPCE.pce_tools as pce_tools

import bcs as bcs
import math
import PyUQTk.bcs as bcs_tools

import pandas as pd

PyMC is required for some of the MCMC postprocessing codes.
Will proceed without, but some convergence tests will not be available.


## Inputs

In [2]:
nord = 5            # order of the polynomial
ndim = 6            # number of dimensions
pc_type="LU"        # type of PC

c_opt = "alt"        # method for selecting starting coefficients

pc_alpha = 0.0      # Free parameter > -1 for Gamma-Laguerre and Beta-Jacobi PCs
pc_beta = 1.0       # Free parameter > -1 for Gamma-Laguerre and Beta-Jacobi PCs

## Initialize PC

In [3]:
pc_model = uqtkpce.PCSet("NISPnoq", nord, ndim, pc_type, pc_alpha, pc_beta)

## BSC Inputs

In [4]:
sigma = np.array([1.e-08]) # inital noise variance; updated in BCS
#eta = 1e-7                 # algorithm stops once evidence gain falls below this threshold
lambda_init = np.array([]) # paramter of the laplace distribution; leave empty or set lambda to a fixed nonnegative value

## Define coefficients

In [5]:
npce=pc_model.GetNumberPCTerms()

# sequence of increasing integers
if (c_opt=="seq"):
    coef=np.array([1/(i+1) for i in range(npce)])
    
# sequence of increasing integers alternating with zero    
elif (c_opt=="alt"):
    coef=np.zeros(npce)
    for i in range(npce):
        if (i%2==0):
            coef[i]=1/(i+1)

In [6]:
rng = qmc.LatinHypercube(d=ndim, seed=42)
nTest=int(npce*.75)
nfolds=2
x=2*rng.random(n=(nTest*nfolds))-1 # [-1,1]
y=pce_tools.UQTkEvaluatePCE(pc_model, coef, x)

In [7]:
e_tr=1/np.power(10,[i for i in range(1,13)])

In [15]:
eta, eta_std = pce_tools.UQTkOptimizeEta(pc_model, x, y, e_tr, nfolds, sigma)

In [16]:
eta

5.05e-10

In [9]:
k=bcs_tools.kfoldCV(x, y, 2)
eta_list=[]
for i in range(nfolds):
    x_tr=k[i]['xtrain']
    y_tr=k[i]['ytrain']
    x_val=k[i]['xval']
    y_val=k[i]['yval']
    E_v=[]
    for j, eta in enumerate(e_tr):
        c_tr =pce_tools.UQTkBCS(pc_model, y_tr, x_tr, sigma, eta, lambda_init)
        phi_v_c_tr=pce_tools.UQTkEvaluatePCE(pc_model, c_tr, x_val)
        E_v.append(math.sqrt(np.square(np.subtract(y_val, phi_v_c_tr)).mean()))
    eta_o = e_tr[E_v.index(np.min(E_v))]
    eta_list.append(e_tr[E_v.index(np.min(E_v))]*math.sqrt(x.shape[0]/x_tr.shape[0]))
    print(eta_o)
eta_opt=np.array(eta_list).mean()
eta_std=np.std(np.array(eta_list))

TypeError: UQTkBCS() takes 5 positional arguments but 6 were given

## Testing Points

In [ ]:
x_test=2*rng.random(n=pc_model.GetNumberPCTerms())-1 # [-1,1]
y_test=pce_tools.UQTkEvaluatePCE(pc_model, coef, x_test)

## Calculate Errors

In [ ]:
c_k, lam =pce_tools.UQTkBCS(pc_model, y_tr, x_tr, sigma, eta_opt, lambda_init)

In [ ]:
MSE = np.square(np.subtract(c_k, coef)).mean()
RMSE=math.sqrt(MSE)
RMSE/np.linalg.norm(y_test)

In [ ]:
pce_evals=pce_tools.UQTkEvaluatePCE(pc_model, c_k, x_test)
MSE = np.square(np.subtract(y_test, pce_evals)).mean()
RMSE=math.sqrt(MSE)
RMSE/np.linalg.norm(y_test)

In [ ]:
print("Nonzero coefficients found:  ", np.nonzero(c_k)[0].shape[0])
print("Total nonzero coefficients:  ", np.nonzero(coef)[0].shape[0])

In [ ]:
pd.DataFrame(data=np.transpose([c_k, coef]), columns=['bcs', 'actual'])